## Deliverable 2. Create a Customer Travel Destinations Map.

In [92]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [93]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Arvika,SE,2021-12-10 17:39:51,59.6553,12.5852,32.70,99,100,7.18,light snow
1,1,Khatanga,RU,2021-12-10 17:35:23,71.9667,102.5000,-34.53,100,100,4.99,overcast clouds
2,2,Bluff,NZ,2021-12-10 17:32:29,-46.6000,168.3333,55.80,79,100,9.66,overcast clouds
3,3,Nanortalik,GL,2021-12-10 17:39:52,60.1432,-45.2371,27.73,79,100,5.37,overcast clouds
4,4,Yajalon,MX,2021-12-10 17:39:53,17.2333,-92.3333,75.02,40,13,3.42,few clouds


In [94]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [95]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Yajalon,MX,2021-12-10 17:39:53,17.2333,-92.3333,75.02,40,13,3.42,few clouds
6,6,Poum,NC,2021-12-10 17:34:04,-20.2333,164.0167,81.77,78,100,13.58,overcast clouds
9,9,Beloha,MG,2021-12-10 17:39:55,-25.1667,45.0500,75.92,79,95,10.29,overcast clouds
12,12,Rikitea,PF,2021-12-10 17:30:56,-23.1203,-134.9692,76.05,73,98,17.69,overcast clouds
19,19,Black River,JM,2021-12-10 17:39:59,18.0264,-77.8487,84.97,55,9,7.52,clear sky
20,20,Atuona,PF,2021-12-10 17:32:34,-9.8000,-139.0333,77.97,73,43,14.00,scattered clouds
25,25,Tadine,NC,2021-12-10 17:40:01,-21.5500,167.8833,78.28,85,98,12.12,overcast clouds
29,29,Vung Tau,VN,2021-12-10 17:38:30,10.3460,107.0843,76.78,76,86,4.68,overcast clouds
48,48,Sao Filipe,CV,2021-12-10 17:41:12,14.8961,-24.4956,76.51,65,41,9.75,scattered clouds
49,49,Hithadhoo,MV,2021-12-10 17:35:33,-0.6000,73.0833,81.41,79,94,9.69,overcast clouds


In [96]:
# 4a. Determine if there are any empty rows.
preferred_df.count()
preferred_df.count().isnull

<bound method Series.isnull of City_ID        202
City           202
Country        198
Date           202
Lat            202
Lng            202
Max Temp       202
Humidity       202
Cloudiness     202
Wind Speed     202
Description    202
dtype: int64>

In [97]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Yajalon,MX,2021-12-10 17:39:53,17.2333,-92.3333,75.02,40,13,3.42,few clouds
6,6,Poum,NC,2021-12-10 17:34:04,-20.2333,164.0167,81.77,78,100,13.58,overcast clouds
9,9,Beloha,MG,2021-12-10 17:39:55,-25.1667,45.0500,75.92,79,95,10.29,overcast clouds
12,12,Rikitea,PF,2021-12-10 17:30:56,-23.1203,-134.9692,76.05,73,98,17.69,overcast clouds
19,19,Black River,JM,2021-12-10 17:39:59,18.0264,-77.8487,84.97,55,9,7.52,clear sky
...,...,...,...,...,...,...,...,...,...,...,...
667,667,Cayenne,GF,2021-12-10 17:57:25,4.9333,-52.3333,84.24,89,75,1.01,broken clouds
670,670,Ipilan,PH,2021-12-10 17:59:35,8.8431,117.9033,79.68,82,85,8.08,overcast clouds
671,671,Manado,ID,2021-12-10 17:59:36,1.4870,124.8455,81.37,89,40,4.61,scattered clouds
674,674,Alibag,IN,2021-12-10 17:59:38,18.6411,72.8792,82.47,66,0,6.29,clear sky


In [98]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Yajalon,MX,75.02,few clouds,17.2333,-92.3333,
6,Poum,NC,81.77,overcast clouds,-20.2333,164.0167,
9,Beloha,MG,75.92,overcast clouds,-25.1667,45.0500,
12,Rikitea,PF,76.05,overcast clouds,-23.1203,-134.9692,
19,Black River,JM,84.97,clear sky,18.0264,-77.8487,
20,Atuona,PF,77.97,scattered clouds,-9.8000,-139.0333,
25,Tadine,NC,78.28,overcast clouds,-21.5500,167.8833,
29,Vung Tau,VN,76.78,overcast clouds,10.3460,107.0843,
48,Sao Filipe,CV,76.51,scattered clouds,14.8961,-24.4956,
49,Hithadhoo,MV,81.41,overcast clouds,-0.6000,73.0833,


In [99]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
    

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [108]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Poum,NC,81.77,overcast clouds,-20.2333,164.0167,Gîte kajeon
12,Rikitea,PF,76.05,overcast clouds,-23.1203,-134.9692,People ThankYou
19,Black River,JM,84.97,clear sky,18.0264,-77.8487,Idlers' Rest Beach Hotel
20,Atuona,PF,77.97,scattered clouds,-9.8000,-139.0333,Villa Enata
29,Vung Tau,VN,76.78,overcast clouds,10.3460,107.0843,Green hotel
48,Sao Filipe,CV,76.51,scattered clouds,14.8961,-24.4956,Hotel Xaguate
49,Hithadhoo,MV,81.41,overcast clouds,-0.6000,73.0833,Scoop Guest House
50,Bima,ID,77.13,overcast clouds,-8.4667,118.7167,Lambitu Hotel
54,Cabo San Lucas,MX,81.52,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
56,Isangel,VU,78.60,broken clouds,-19.5500,169.2667,Tanna Lodge


In [113]:
# 8a. Create the output File (CSV)
output_hotel_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_hotel_file, index_label="City_ID")

In [116]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</d
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [120]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))